In [1]:
import sys
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.activations import sigmoid
from keras.activations import relu
from keras.activations import tanh
from keras.losses import MSE
from keras.losses import mean_squared_error
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.metrics import binary_accuracy
from tensorflow.compat.v1.graph_util import convert_variables_to_constants
import os

Using TensorFlow backend.


In [2]:
sess=tf.Session()
K.set_session(sess)

In [3]:
training_data = np.array([[0,0], [0,1], [1,0], [1,1]])
target_data   = np.array([  [0],   [1],   [1],   [0]])

In [4]:
model = Sequential()
model.add(Dense(4, input_dim=2, activation='sigmoid'))
#model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer=SGD(lr=1), metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.1), metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
epochs = 2000
model.fit(training_data, target_data, verbose=0, epochs=epochs)

Instructions for updating:
Use tf.cast instead.


In [6]:
print ("loss:", model.evaluate(x=training_data, y=target_data, verbose=0))
print ("")
print (model.predict(training_data))

loss: [0.00184284383431077, 1.0]

[[0.02486864]
 [0.9554728 ]
 [0.95746845]
 [0.05441805]]


In [7]:
print (model.predict(training_data)[2], "<<<<<")

[0.95746845] <<<<<


## Save
### freeze model and save as Tensorflow 

In [8]:
### freeze model and save as Tensorflow 

freeze_var_names = list(set(v.op.name for v in tf.global_variables()))
output_names = [model.outputs[0].name.split(":")[0]]

frozen_graph = convert_variables_to_constants(sess,
                                              sess.graph.as_graph_def(),
                                              output_names,
                                              freeze_var_names)

graph_path = tf.train.write_graph(frozen_graph, "models", "xor.pb", as_text=False)
print ("")
print ("%s written" % graph_path)
print (model.inputs[0].name)
print (model.outputs[-1].name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.

models/xor.pb written
dense_1_input:0
dense_2/Sigmoid:0


### save as a keras h5 model

In [9]:
h5_file = "models/xor.h5"
model.save(h5_file)
print ("%s written" % h5_file)

models/xor.h5 written
